# 네이버 매장정보 크롤링

#### 자료 
https://www.localdata.go.kr/devcenter/dataDown.do?menuNo=20001

1. 관광식당, 일반음식점, 휴게음식점 데이터셋 다운로드(07_24_01_P_.csv.zip, 07_24_04_P_.csv.zip , 07_24_05_P_.csv.zip)
2. 다운로드한 파일들을 data 폴더에 넣어주세요. (압축풀지말고)

문제 
1. 강남구 매장이지만, 네이버로 검색하는 경우 강남구가 아닌 다른 지역의 매장명이 검색됨
2. 데이터 반영이 빠른편이지만, 네이버에 매장이 없는 경우가 있음 (없는 매장 : 맥도날드, 교촌, 굽네) 
3. 검색하면 전혀 다른 업체가 나오는 경우가 있음 -> 네이버 업태구분명을 크롤링해와서 참고하여 반영해야 
4. 나라에서 제공하는 data를 사용하는 경우 인기도를 알기 어려움. 아닌가. -> 네이버 지도 api를 사용해서 점수를 부여할까? 
5. 자료 업데이트 시기 문제 - data를 다운로드 받아서 사용하는 경우, 매장 정보는 계속 바뀐는데 업데이트가 늦잖아... 매장 정보의 변동이 생기 사람이 계속 업로드를 해야 하잖아.. 

In [3]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import zipfile
import os


In [4]:
# 지역 설정
city = '서울특별시'
gu = '강남구'

In [5]:
# zip 파일 압축 풀기 

folder_path = './data'
zip_folder = './data/zip_data'

# data 폴더 안에 있는 압축 파일들 압축 풀기
for file in os.listdir(folder_path):
    zip_file_path = os.path.join(folder_path, file)
    if zipfile.is_zipfile(zip_file_path): 
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(zip_folder)

# csv 파일이 큰 순서대로 정렬
csv_files_sorted = sorted([os.path.join(zip_folder, f) for f in os.listdir(zip_folder) if f.endswith('.csv')],
                          key=lambda x: os.path.getsize(x),reverse=True)
csv_files_sorted

['./data/zip_data\\fulldata_07_24_04_P_└╧╣▌└╜╜─┴í.csv',
 './data/zip_data\\fulldata_07_24_05_P_╚▐░╘└╜╜─┴í.csv',
 './data/zip_data\\fulldata_07_24_01_P_░ⁿ▒ñ╜─┤τ.csv']

In [6]:
# 첫번째 데이터 불러오기 (데이터 크기가 커서 둘로 나눈뒤 합침)
chunk_size = 1000000
chunks = pd.read_csv(csv_files_sorted[0], encoding='cp949', encoding_errors='ignore', chunksize=chunk_size)
df1_1 = next(chunks)  # 첫 번째 청크(1,000,000 행) 읽기
df1_2 = pd.concat(chunks, ignore_index=True)  # 나머지 데이터 읽기

# 현재 운영하고 있는 매장만 가져와서 합치기 
df1_1.drop(df1_1[df1_1['영업상태명'] == '폐업'].index, inplace=True)
df1_2.drop(df1_2[df1_2['영업상태명'] == '폐업'].index, inplace=True)
df1 = pd.concat([df1_1, df1_2], ignore_index=True)


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 695479 entries, 0 to 695478
Data columns (total 48 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   번호           695479 non-null  int64  
 1   개방서비스명       695479 non-null  object 
 2   개방서비스아이디     695479 non-null  object 
 3   개방자치단체코드     695479 non-null  int64  
 4   관리번호         695479 non-null  object 
 5   인허가일자        695473 non-null  object 
 6   인허가취소일자      0 non-null       float64
 7   영업상태구분코드     695479 non-null  int64  
 8   영업상태명        695479 non-null  object 
 9   상세영업상태코드     695479 non-null  int64  
 10  상세영업상태명      695479 non-null  object 
 11  폐업일자         0 non-null       object 
 12  휴업시작일자       0 non-null       float64
 13  휴업종료일자       0 non-null       float64
 14  재개업일자        0 non-null       float64
 15  소재지전화        371413 non-null  object 
 16  소재지면적        680002 non-null  float64
 17  소재지우편번호      612656 non-null  object 
 18  소재지전체주소      694077 non-

In [8]:
df1.columns

Index(['번호', '개방서비스명', '개방서비스아이디', '개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자',
       '영업상태구분코드', '영업상태명', '상세영업상태코드', '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자',
       '재개업일자', '소재지전화', '소재지면적', '소재지우편번호', '소재지전체주소', '도로명전체주소', '도로명우편번호',
       '사업장명', '최종수정시점', '데이터갱신구분', '데이터갱신일자', '업태구분명', '좌표정보(x)', '좌표정보(y)',
       '위생업태명', '남성종사자수', '여성종사자수', '영업장주변구분명', '등급구분명', '급수시설구분명', '총직원수',
       '본사직원수', '공장사무직직원수', '공장판매직직원수', '공장생산직직원수', '건물소유구분명', '보증액', '월세액',
       '다중이용업소여부', '시설총규모', '전통업소지정번호', '전통업소주된음식', '홈페이지', 'Unnamed: 47'],
      dtype='object')

In [9]:
# 두번째 데이터 불러오기 
df2 = pd.read_csv(csv_files_sorted[1], encoding='cp949', encoding_errors='ignore')
df2.columns

Index(['번호', '개방서비스명', '개방서비스아이디', '개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자',
       '영업상태구분코드', '영업상태명', '상세영업상태코드', '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자',
       '재개업일자', '소재지전화', '소재지면적', '소재지우편번호', '소재지전체주소', '도로명전체주소', '도로명우편번호',
       '사업장명', '최종수정시점', '데이터갱신구분', '데이터갱신일자', '업태구분명', '좌표정보(x)', '좌표정보(y)',
       '위생업태명', '남성종사자수', '여성종사자수', '영업장주변구분명', '등급구분명', '급수시설구분명', '총직원수',
       '본사직원수', '공장사무직직원수', '공장판매직직원수', '공장생산직직원수', '건물소유구분명', '보증액', '월세액',
       '다중이용업소여부', '시설총규모', '전통업소지정번호', '전통업소주된음식', '홈페이지', 'Unnamed: 47'],
      dtype='object')

In [10]:
# 세번째 데이터 불러오기 
df3 = pd.read_csv(csv_files_sorted[2], encoding='cp949', encoding_errors='ignore')
df3.columns

Index(['번호', '개방서비스명', '개방서비스아이디', '개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자',
       '영업상태구분코드', '영업상태명', '상세영업상태코드', '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자',
       '재개업일자', '소재지전화', '소재지면적', '소재지우편번호', '소재지전체주소', '도로명전체주소', '도로명우편번호',
       '사업장명', '최종수정시점', '데이터갱신구분', '데이터갱신일자', '업태구분명', '좌표정보(x)', '좌표정보(y)',
       '문화체육업종명', '문화사업자구분명', '지역구분명', '총층수', '주변환경명', '제작취급품목내용', '보험기관명',
       '건물용도명', '지상층수', '지하층수', '객실수', '건축연면적', '영문상호명', '영문상호주소', '선박총톤수',
       '선박척수', '선박제원', '무대면적', '좌석수', '기념품종류', '회의실별동시수용인원', '시설면적', '놀이기구수내역',
       '놀이시설수', '방송시설유무', '발전시설유무', '의무실유무', '안내소유무', '기획여행보험시작일자',
       '기획여행보험종료일자', '자본금', '보험시작일자', '보험종료일자', '부대시설내역', '시설규모',
       'Unnamed: 63'],
      dtype='object')

In [11]:
# 필요한 컬럼만 들고 오기 
selected_columns = ['영업상태명', '소재지전체주소', '도로명전체주소', '사업장명', '최종수정시점','데이터갱신일자','업태구분명', '좌표정보(x)', '좌표정보(y)','위생업태명']

df1 = df1[selected_columns]

# 지역구 구분하기 
df1['시도'] = df1['소재지전체주소'].apply(lambda x: x.split(' ', 1)[0] if pd.notna(x) else None)
df1['지역구'] = df1['소재지전체주소'].apply(lambda x: x.split(' ')[1] if pd.notna(x) and len(x.split(' ')) > 1 else None)



# 현재 운영하고 있는 매장만 가져오기
df2.drop(df2[df2['영업상태명'] == '폐업'].index, inplace=True)

# 필요한 컬럼만 들고 오기 
df2 = df2[selected_columns]

# 지역구 구분하기 
df2['시도'] = df2['소재지전체주소'].apply(lambda x: x.split(' ', 1)[0] if pd.notna(x) else None)
df2['지역구'] = df2['소재지전체주소'].apply(lambda x: x.split(' ')[1] if pd.notna(x) and len(x.split(' ')) > 1 else None)


# 현재 운영하고 있는 매장만 가져오기
df3.drop(df3[df3['영업상태명'] == '폐업'].index, inplace=True)

# 필요한 컬럼만 들고 오기 
df3 = df1[selected_columns]

# 지역구 구분하기 
df3['시도'] = df3['소재지전체주소'].apply(lambda x: x.split(' ', 1)[0] if pd.notna(x) else None)
df3['지역구'] = df3['소재지전체주소'].apply(lambda x: x.split(' ')[1] if pd.notna(x) and len(x.split(' ')) > 1 else None)

In [12]:
# 데이터 합치고 중복제거
origin_df = pd.concat([df1, df2, df3], ignore_index=True).drop_duplicates()
origin_df.head(10)

,영업상태명,소재지전체주소,도로명전체주소,사업장명,최종수정시점,데이터갱신일자,업태구분명,좌표정보(x),좌표정보(y),위생업태명,시도,지역구
0,영업/정상,경기도 하남시 선동 427 미사강변 더샵 리버포레,"경기도 하남시 미사강변북로 65-1, 미사강변 더샵 리버포레 1층 101호 (선동)",33떡볶이&꼬마김밥 미사리버포레점,2023-11-30 15:09:25,2023-12-02 00:16:59,분식,216355.623702,452406.828698,분식,경기도,하남시
1,영업/정상,"인천광역시 중구 운서동 3094-2 더 예스 포레스트 2층 205,206호","인천광역시 중구 흰바위로 113, 더 예스 포레스트 2층 205,206호 (운서동)",중화요리 미식재,2023-12-22 11:09:11,2023-12-24 02:40:00,중국식,155439.523172,443179.669702,중국식,인천광역시,중구
2,영업/정상,부산광역시 동래구 온천동 96-5,"부산광역시 동래구 금강공원로26번길 29, 2층 일부호 (온천동)",웨이브라운지 비어,2023-11-30 15:10:28,2023-12-02 00:16:59,호프/통닭,389423.885905,193211.891622,호프/통닭,부산광역시,동래구
3,영업/정상,부산광역시 동래구 낙민동 326 포레나 동래 201동 7호,"부산광역시 동래구 온천천로339번길 51, 201동 1층 7호 (낙민동, 포레나 동래)",4242샌드위치 동래점,2023-11-30 14:15:09,2023-12-02 00:16:59,경양식,390304.544719,190552.782902,경양식,부산광역시,동래구
4,영업/정상,충청북도 증평군 증평읍 송산리 881,충청북도 증평군 증평읍 송산로5길 37,백세장수촌 증평점,2023-11-30 16:44:21,2023-12-02 00:16:59,한식,252106.125904,366017.067263,한식,충청북도,증평군
5,영업/정상,경상남도 양산시 동면 석산리 1453-4,"경상남도 양산시 동면 금오16길 12, 102호",스시모리,2023-11-24 15:27:19,2023-11-26 00:17:03,기타,384239.479031,203911.027778,기타,경상남도,양산시
6,영업/정상,충청남도 금산군 금산읍 상리 89-4 중부프라자,"충청남도 금산군 금산읍 비범로 19, 1층 B-5호",금주네,2023-11-24 16:45:09,2023-11-26 00:17:03,한식,244133.098932,289437.65349,한식,충청남도,금산군
7,영업/정상,경기도 안산시 상록구 수암동 499-1 1층 101호,"경기도 안산시 상록구 원당골1길 6, 1층 101호 (수암동)",국풍설렁탕(수암본점),2023-11-24 16:23:55,2023-11-26 00:17:03,한식,189070.433282,428983.284759,한식,경기도,안산시
8,영업/정상,경기도 안산시 상록구 본오동 873-7 리치스빌딩 1층 일부,"경기도 안산시 상록구 본삼로 51, 리치스빌딩 1층 일부 (본오동)",심야스테이크 상록수점,2023-11-24 14:37:22,2023-11-26 00:17:03,경양식,188039.338430,422128.686119,경양식,경기도,안산시
9,영업/정상,서울특별시 마포구 서교동 346-34,"서울특별시 마포구 와우산로23길 48, 2,3층 (서교동)",(주)소과당,2023-11-24 16:04:20,2023-11-26 00:17:03,기타,193189.149801,450296.922232,기타,서울특별시,마포구


In [13]:
origin_df = pd.read_csv('./data/강남구_매장명.csv',encoding='cp949', encoding_errors='ignore')
origin_df

,영업상태명,소재지전체주소,도로명전체주소,사업장명,최종수정시점,데이터갱신일자,업태구분명,좌표정보(x),좌표정보(y),위생업태명,시도,지역구,주소_매장명
0,영업/정상,서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",정식당,45191.48189,45193.01247,한식,202203.9718,444853.7615,한식,서울특별시,강남구,강남대로114길 18 정식당
1,영업/정상,서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",채선당 샤브보트 강남역점,45191.43059,45193.01247,한식,202462.7943,443827.9619,한식,서울특별시,강남구,강남대로 378 채선당 샤브보트 강남역점
2,영업/정상,서울특별시 강남구 삼성동 159 코엑스,"서울특별시 강남구 영동대로 513, 코엑스 C홀 지상3층 (삼성동)",(주)푸들,45261.74914,45263.01222,기타,205130.5917,445590.0968,기타,서울특별시,강남구,영동대로 513 (주)푸들
3,영업/정상,서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",수아당,45261.70965,45263.01222,한식,203470.8484,447369.5799,한식,서울특별시,강남구,압구정로 343 수아당
4,영업/정상,서울특별시 강남구 삼성동 116-10,"서울특별시 강남구 봉은사로72길 11, 지하1층 (삼성동)",오늘은 철판,45261.66803,45263.01222,일식,204447.4012,445478.0666,일식,서울특별시,강남구,봉은사로72길 11 오늘은 철판
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16379,영업/정상,서울특별시 강남구 논현동 142-2 JS타워 지상1층,"서울특별시 강남구 강남대로 538, JS타워 지상1층 (논현동)",스타벅스 논현역사거리점,44977.65293,44979.11111,커피숍,201908.7511,445336.4638,커피숍,서울특별시,강남구,강남대로 538 스타벅스 논현역사거리점
16380,영업/정상,서울특별시 강남구 논현동 119 포바강남타워,"서울특별시 강남구 학동로 343, 포바강남타워 지하1층 B103-1호 (논현동)",켈리 토스트 카페 강남구청역점,45188.42432,45190.11111,패스트푸드,203522.6117,446136.2250,패스트푸드,서울특별시,강남구,학동로 343 켈리 토스트 카페 강남구청역점
16381,영업/정상,서울특별시 강남구 신사동 664-6,"서울특별시 강남구 선릉로 845, 지상1층 6호 (신사동)",커피에 반하다,44910.65238,44912.11111,커피숍,203493.4986,447259.6515,커피숍,서울특별시,강남구,선릉로 845 커피에 반하다
16382,영업/정상,서울특별시 강남구 논현동 218번지 지상1층,"서울특별시 강남구 언주로125길 7, 지상1층 (논현동)",CU 논현세원점,43409.45632,43411.10948,편의점,202990.9797,445632.5812,편의점,서울특별시,강남구,언주로125길 7 CU 논현세원점


In [14]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install())
# 주소 이동
url = 'https://map.naver.com/'
driver.get(url)
time.sleep(1)

# 매장명이 있나없나 테스트

In [15]:
name = '강남대로114길 18 정식당'
# 검색어를 네이버 url에 포함시켜 이동 + 검색 
driver.get('https://map.naver.com/p/search/{}'.format(name))

# 저장 
naver_df = pd.DataFrame()
time.sleep(3)  

searchIframe = driver.find_element(By.ID,'searchIframe')
driver.switch_to.frame(searchIframe)
time.sleep(3) 

In [16]:
점검 = pd.DataFrame

for name in origin_df['주소_매장명'] :
    print(name)
    # 검색어를 네이버 url에 포함시켜 이동 + 검색 
    driver.get('https://map.naver.com/p/search/{}'.format(name))

    # 저장 
    naver_df = pd.DataFrame()
    time.sleep(3)  

    searchIframe = driver.find_element(By.ID,'searchIframe')
    driver.switch_to.frame(searchIframe)
    time.sleep(3) 

강남대로114길 18 정식당
강남대로 378 채선당 샤브보트 강남역점
영동대로 513 (주)푸들


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="searchIframe"]"}
  (Session info: chrome=121.0.6167.161)


In [ ]:
# # 폴더 안에 있는 파일을 반복해서 df에 저장하고 싶었는데... 메모리 이슈 실패 

# zip_folder = './data/zip_data'
# chunk_size = 1000000
# df_list = []
# n = 1


# for file in os.listdir(zip_folder):
#     file_path = os.path.join(zip_folder, file)
    
#     # 파일 크기가 800MB 이상인지 확인
#     if os.path.getsize(file_path) >= 800000000:
#         chunks = pd.read_csv(file_path, encoding='cp949', encoding_errors='ignore', chunksize=chunk_size)
#         df1_1 = next(chunks)  # 첫 번째 청크(1,000,000 행) 읽기
#         df1_2 = pd.concat(chunks, ignore_index=True)  # 나머지 데이터 읽기

#         # 현재 운영하고 있는 매장만 가져오기
#         df1_1.drop(df1_1[df1_1['영업상태명'] == '폐업'].index, inplace=True)
#         df1_2.drop(df1_2[df1_2['영업상태명'] == '폐업'].index, inplace=True)
#         df_0 = pd.concat([df1_1, df1_2], ignore_index=True)
#         df_list.append('df_0')

#     else:
#         for n in range(1, 4):
#             df_name = 'df_{}'.format(n)
#             globals()[df_name] = pd.read_csv(file_path, encoding='cp949', encoding_errors='ignore')
#             df_list.append(globals()[df_name])
                
# df_list

In [ ]:
# '시도' 컬럼이 '서울특별시'면서 '지역구' 값이 '강남구'인 행만 필터링
df = origin_df[(origin_df['시도'] == city ) & (origin_df['지역구'] == gu )]
df.head()

# csv 파일로 저장
df.to_csv('filtered_data_{}{}.csv'.format(city,gu), index=False)


In [ ]:
df.head(5)

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
# 업태구분명 확인
unique_values = df['업태구분명'].unique()
unique_values

In [ ]:
# 카페, 술, 출장요리 류 삭제
제외 = ['전통찻집','호프/통닭','뷔페식','출장조리','정종/대포집/소주방','이동조리', '감성주점','까페','라이브카페','키즈카페','커피숍','편의점', '일반조리판매','아이스크림', '떡카페', '철도역구내', '푸드트럭', '과자점', '다방', '관광호텔']
for i in 제외 :
    df = df.drop(df[df['업태구분명'] == i].index)
df

In [ ]:
# Test할 매장
df_top10 = df.head(10)
df_top10

# 셀레니움

In [ ]:
# place는 매장명, count는 검색했을때 조회하는 범위
def naver_finder(place,count) :

  # 검색어를 네이버 url에 포함시켜 이동 + 검색 
  driver.get('https://map.naver.com/p/search/{}'.format(place))

  # 저장 
  naver_df = pd.DataFrame()
  time.sleep(3)  

  try : 
    # frame을 3갤 나눔 searchIframe(왼쪽), entryIframe(오른쪽),default_content(기본)

    # 왼쪽 프레임
    searchIframe = driver.find_element(By.ID,'searchIframe')
    driver.switch_to.frame(searchIframe)

    try : 
      # {}안에 num 입력 (1번째, 2번째...)
      # 각 카드 상단을 클릭하여 WebDriverWait을 활용 카드 상단 XPATH가 보일때까지 3초 대기 
      driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li[{}]/div[1]/a'.format(num)).click()

    except :
      # 가능하지 않다면 스크롤 내리고 최대 3초까지 기다렸다가 클릭
      # tag 내용이 안 보이면 해당 tag에서 스크롤 내림
      # click()이 안되는 경우 클릭을 대체하여 사용하기 
      driver.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)
      card_clik = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li[{}]/div[1]/a'.format(num))
      driver.execute_script('arguments[0].click();',card_clik)

    # 기본 프레임
    time.sleep(1)
    driver.switch_to.default_content()

    # 오른쪽 프레임
    entryIframe = driver.find_element(By.ID,'entryIframe')
    driver.switch_to.frame(entryIframe)

    # 가게 이름, 변수로 지정
    restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text

    # 가게 별점이 있는 경우에만 try 
    try :
      review_star = driver.find_element(By.CLASS_NAME,'PXMot').text
      review_star = re.sub('방문자리뷰',"",review_star)
    except :
      review_star = 0
      pass 

    # 기본 설정으로 돌아오기
    time.sleep(1)
    driver.switch_to.default_content()

    print(restaurant_name,review_star)
    
    # 수정 필요 
    df = pd.DataFrame({'가게명':[restaurant_name],'네이버별점' : [review_star]})
    naver_df = pd.concat([naver_df,df])
    naver_df.drop_duplicates() # 중복 제거 
  except :
    print('정상적으로 작동이 되지 않습니다.')

    return naver_df


In [ ]:
for i in df_top10['사업장명']:
    df_sample = naver_finder(i,2)
    time.sleep(5)

정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.
하늘사다리 별점
4.43
은희네 온집닭떡볶이 상도본점 별점
4.47
정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.


""


""
